In [ ]:
import os
import json
import copy
import pywt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

### Config

In [ ]:
target_var = "onset"   # 'add', 'onset', or 'hist'

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

BATCH_SIZE_ENC = 64       # CNN+BiLSTM encoder batch size
EPOCHS_ENC = 40           # encoder training epochs
LR_ENC = 1e-3             # encoder learning rate

D_MODEL = 64              # CNN/BiLSTM feature dim
GA_EMB_DIM = 16           # GA MLP embedding dim
DROPOUT = 0.20

WAVELET_NAME = "db4"
WAVELET_LEVEL = 3         # raw + D1..D3 + A3 = 5 channels

N_FOLDS = 5

OUT_DIR = "/content/drive/MyDrive/ts2vec-05-yuntao"
os.makedirs(OUT_DIR, exist_ok=True)

np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
########## For Colab ##########
!pip install ts2vec
from ts2vec import TS2Vec

########## Personal ##########
from google.colab import drive
drive.mount('/content/drive')
with open(f'/content/drive/MyDrive/datasets/dataset_{target_var}.json') as f:
    content = f.read()
    data = json.loads(content)

########## Enterprise ##########
# import gcsfs
# fs = gcsfs.GCSFileSystem()
# with fs.open('gs://modoo-eod/users/datasets/dataset_hist.json') as f:
#     content = f.read()
#     data = json.loads(content)

######### Local ##########
# with open("../datasets/dataset_onset.json") as f:
#     content=f.read()
#     data=json.loads(content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.DataFrame.from_records(data)
print(len(df), "Measurements (raw)")

3681 Measurements (raw)


### Basic Cleaning & Static Imputation

In [ ]:
age_mean = np.mean([i[0] for i in df["static"] if pd.notna(i[0])])
bmi_mean = np.mean([i[1] for i in df["static"] if pd.notna(i[1])])

cleaned_data = []
for _, m in enumerate(data):

    # Remove measurements with empty windows
    if len(m["uc_windows"]) == 0 or len(m["fhr_windows"]) == 0:
        continue

    # Handle NaN values in static
    static = m["static"].copy()
    if pd.isna(static[0]):
        static[0] = age_mean
    if pd.isna(static[1]):
        static[1] = bmi_mean

    copy_m = m.copy()
    copy_m["static"] = static
    cleaned_data.append(copy_m)

cleaned_df = pd.DataFrame(cleaned_data)
print(len(cleaned_df), "Cleaned Measurements")

# 假定 static 最后一个元素是 gest_age_days
cleaned_df["gest_age_weeks"] = [(i[-1] // 7) + 1 for i in cleaned_df["static"]]

base_records = cleaned_df.to_dict(orient="records")  # 原始不带任何 fold 修改的版本
N = len(base_records)
all_weeks = cleaned_df["gest_age_weeks"].to_numpy()

print("Gestational age weeks distribution:")
print(cleaned_df["gest_age_weeks"].value_counts().sort_index())

3661 Cleaned Measurements
Gestational age weeks distribution:
gest_age_weeks
29     25
30     72
31    105
32    157
33    254
34    313
35    371
36    443
37    503
38    544
39    463
40    321
41     89
42      1
Name: count, dtype: int64


### SWT Helper

In [ ]:
def sproc_wavelet_channels(x_1d: np.ndarray):
    """
    SWT(db4, level=3) -> 通道: [raw, D1, D2, D3, A3]
    """
    x_1d = np.asarray(x_1d, dtype=np.float32)
    coeffs = pywt.swt(
        x_1d, wavelet=WAVELET_NAME,
        level=WAVELET_LEVEL,
        trim_approx=False,
        norm=True
    )
    # coeffs[l] = (cA_l, cD_l)
    chans = [x_1d.astype(np.float32)]  # raw
    for l in range(WAVELET_LEVEL):
        cA, cD = coeffs[l]
        chans.append(cD.astype(np.float32))   # D1..D3
    chans.append(coeffs[-1][0].astype(np.float32))  # A3
    X = np.stack(chans, axis=0)  # (C, T)

    # 逐通道 robust 标准化
    X = (X - np.median(X, axis=1, keepdims=True)) / (
        np.std(X, axis=1, keepdims=True) + 1e-6
    )
    return X.astype(np.float32)

### CNN+BiLSTM + GA encoder

In [ ]:
class SeqGADataset(Dataset):
    """
    Dataset for single signal (uc_raw or fhr_raw) + GA + target
    在 __getitem__ 里做 SWT，输出 (C, T)。
    """
    def __init__(self, measurements, signal_key):
        self.measurements = measurements
        self.signal_key = signal_key

    def __len__(self):
        return len(self.measurements)

    def __getitem__(self, idx):
        m = self.measurements[idx]

        seq = np.asarray(m[self.signal_key], dtype=np.float32)  # (T,)
        x_ch = sproc_wavelet_channels(seq)                       # (C, T)

        y = np.float32(m["target"])
        ga = np.float32(m["gest_age_weeks"])  # gest age in weeks

        return torch.from_numpy(x_ch), torch.tensor(y), torch.tensor(ga)


class ConvBlock(nn.Module):
    def __init__(self, c_in, c_out, k, d=1, p=0.0, groups_gn=8):
        super().__init__()
        pad = (k - 1) // 2 * d
        self.conv = nn.Conv1d(
            c_in, c_out, kernel_size=k, dilation=d,
            padding=pad, bias=False
        )
        self.gn = nn.GroupNorm(num_groups=min(groups_gn, c_out),
                               num_channels=c_out)
        self.act = nn.SiLU()
        self.dp = nn.Dropout(p)
        self.res = (c_in == c_out)

    def forward(self, x):
        y = self.dp(self.act(self.gn(self.conv(x))))
        return y + x if self.res else y


class CNNBiLSTM_GA_Encoder(nn.Module):
    """
    结构参考你原来的 CNNBiLSTM_GA_Reg：
    - stem Conv1d
    - 多层 ConvBlock (dilation k=9,7,5, d=1,2,4)
    - BiLSTM
    - GA MLP
    - 输出:
        pred: (B,)
        feat: (B, D_MODEL + GA_EMB_DIM)
    输入:
        x:  (B, C, T)  SWT 通道
        ga: (B,)
    """
    def __init__(self, in_ch=5, d_model=64, ga_emb_dim=16, dropout=0.2):
        super().__init__()
        self.d_model = d_model
        self.ga_emb_dim = ga_emb_dim

        self.stem = nn.Sequential(
            nn.Conv1d(in_ch, d_model, kernel_size=7, padding=3, bias=False),
            nn.GroupNorm(num_groups=8, num_channels=d_model),
            nn.SiLU(),
        )

        self.cnn_blocks = nn.Sequential(
            ConvBlock(d_model, d_model, k=9, d=1, p=dropout),
            ConvBlock(d_model, d_model, k=7, d=2, p=dropout),
            ConvBlock(d_model, d_model, k=5, d=4, p=dropout),
            nn.Conv1d(d_model, d_model, kernel_size=1, bias=False),
            nn.SiLU(),
            nn.Dropout(dropout),
        )

        self.bilstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model // 2,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )

        self.pre_pool_dp = nn.Dropout(dropout)

        self.ga_mlp = nn.Sequential(
            nn.Linear(1, 16),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(16, ga_emb_dim),
            nn.SiLU(),
        )

        fuse_dim = d_model + ga_emb_dim
        self.head = nn.Sequential(
            nn.Linear(fuse_dim, fuse_dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(fuse_dim, 1),
        )

    def forward(self, x, ga_scalar):
        """
        x:  (B, C, T)
        ga_scalar: (B,)
        """
        f = self.stem(x)            # (B, d_model, T)
        f = self.cnn_blocks(f)      # (B, d_model, T)

        # BiLSTM
        f_t = f.transpose(1, 2)     # (B, T, d_model)
        y, _ = self.bilstm(f_t)     # (B, T, d_model)
        y = self.pre_pool_dp(y)

        # 时间维 average pooling
        feat_seq = y.mean(dim=1)    # (B, d_model)

        # GA embedding
        ga_emb = self.ga_mlp(ga_scalar.unsqueeze(1))  # (B, ga_emb_dim)

        fused = torch.cat([feat_seq, ga_emb], dim=1)  # (B, d_model + ga_emb_dim)
        pred = self.head(fused).squeeze(-1)           # (B,)

        return pred, fused


def train_ga_encoder(measurements, signal_key):
    """
    在 train 集上训练一个 SWT+CNN+BiLSTM+GA encoder（监督回归），返回训练好的模型。
    """
    dataset = SeqGADataset(measurements, signal_key)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE_ENC,
                        shuffle=True, drop_last=False)

    model = CNNBiLSTM_GA_Encoder(
        in_ch=5,
        d_model=D_MODEL,
        ga_emb_dim=GA_EMB_DIM,
        dropout=DROPOUT,
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR_ENC)
    loss_fn = nn.MSELoss()

    model.train()
    for epoch in range(1, EPOCHS_ENC + 1):
        epoch_loss = 0.0
        n_samples = 0
        for x_ch, y, ga in loader:
            x_ch = x_ch.to(DEVICE)      # (B, C, T)
            y = y.to(DEVICE)            # (B,)
            ga = ga.to(DEVICE)          # (B,)

            optimizer.zero_grad()
            pred, _ = model(x_ch, ga)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()

            bs = x_ch.size(0)
            epoch_loss += loss.item() * bs
            n_samples += bs

        print(f"[{signal_key}] Epoch {epoch:03d} | Loss = {epoch_loss / n_samples:.4f}")

    return model


def extract_ga_features(measurements, signal_key, model):
    """
    用训练好的 encoder 抽取融合 GA 的特征。
    返回 (N, d_model + ga_emb_dim) 的 numpy 数组。
    """
    dataset = SeqGADataset(measurements, signal_key)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE_ENC,
                        shuffle=False, drop_last=False)

    model.eval()
    feats = []
    with torch.no_grad():
        for x_ch, y, ga in loader:
            x_ch = x_ch.to(DEVICE)       # (B, C, T)
            ga = ga.to(DEVICE)
            _, feat = model(x_ch, ga)    # (B, d_model + ga_emb_dim)
            feats.append(feat.cpu().numpy())

    feats = np.concatenate(feats, axis=0)
    return feats

### 5-Fold Cross-Validation

In [ ]:
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

all_true = []
all_pred = []
all_weeks_pred = []
fold_metrics = []
rows_week_fold = []
all_pred_rows = []

indices = np.arange(N)
patients = list(set([i['mobile'] for i in base_records]))

for fold, (tr_idx, te_idx) in enumerate(kf.split(np.arange(len(patients))), start=1):

    print(f"\n===== Fold {fold}/{N_FOLDS} =====")

    # 每个 fold 用 base_records 的深拷贝，避免之前的修改污染

    train_patients = [] ; test_patients = []
    for i in tr_idx:
        train_patients.append(patients[i])
    for i in te_idx:
        test_patients.append(patients[i])

    train_recs = [] ; test_recs = []
    for p in train_patients:
        for m in base_records:
            if m['mobile'] == p:
                train_recs.append(copy.deepcopy(m))
    for p in test_patients:
        for m in base_records:
            if m['mobile'] == p:
                test_recs.append(copy.deepcopy(m))

    print(f"Train : {len(train_recs)} from {len(train_patients)} patients")
    print(f"Test  : {len(test_recs)} from {len(test_patients)} patients")

    # train_recs = [copy.deepcopy(base_records[i]) for i in tr_idx]
    # test_recs = [copy.deepcopy(base_records[i]) for i in te_idx]

    # print(f"Train samples: {len(train_recs)}, Test samples: {len(test_recs)}")

    # --- 1. 训练两个 encoder ---
    print("\n[Fold %d] Train encoder for uc_raw" % fold)
    encoder_uc = train_ga_encoder(train_recs, "uc_raw")

    print("\n[Fold %d] Train encoder for fhr_raw" % fold)
    encoder_fhr = train_ga_encoder(train_recs, "fhr_raw")

    # --- 2. 提取 embedding ---
    print("\n[Fold %d] Extract embeddings (uc_raw & fhr_raw)" % fold)
    train_uc_embed = extract_ga_features(train_recs, "uc_raw", encoder_uc)
    train_fhr_embed = extract_ga_features(train_recs, "fhr_raw", encoder_fhr)
    test_uc_embed = extract_ga_features(test_recs, "uc_raw", encoder_uc)
    test_fhr_embed = extract_ga_features(test_recs, "fhr_raw", encoder_fhr)

    for i, e in enumerate(train_uc_embed):
        train_recs[i]["uc_raw"] = e
    for i, e in enumerate(train_fhr_embed):
        train_recs[i]["fhr_raw"] = e
    for i, e in enumerate(test_uc_embed):
        test_recs[i]["uc_raw"] = e
    for i, e in enumerate(test_fhr_embed):
        test_recs[i]["fhr_raw"] = e

    # --- 3. 聚合 window-level 特征 ---
    for rec in train_recs:
        uc_ws = rec["uc_windows"]
        fhr_ws = rec["fhr_windows"]
        # 原始是 list[dict]；这里才做 mean pooling
        uc_w = np.array([[v for _, v in w.items()] for w in uc_ws],
                        dtype=np.float32)
        fhr_w = np.array([[v for _, v in w.items()] for w in fhr_ws],
                         dtype=np.float32)
        rec["uc_windows"] = uc_w.mean(axis=0)
        rec["fhr_windows"] = fhr_w.mean(axis=0)

    for rec in test_recs:
        uc_ws = rec["uc_windows"]
        fhr_ws = rec["fhr_windows"]
        uc_w = np.array([[v for _, v in w.items()] for w in uc_ws],
                        dtype=np.float32)
        fhr_w = np.array([[v for _, v in w.items()] for w in fhr_ws],
                         dtype=np.float32)
        rec["uc_windows"] = uc_w.mean(axis=0)
        rec["fhr_windows"] = fhr_w.mean(axis=0)

    # 维度信息
    FUSED_DIM = len(train_recs[0]["uc_raw"])        # 64 + 16 = 80
    STATIC_DIM = len(train_recs[0]["static"])
    UC_WIN_DIM = len(train_recs[0]["uc_windows"])   # 20
    FHR_WIN_DIM = len(train_recs[0]["fhr_windows"]) # 24

    FEATURE_ORDER = ["uc_raw", "fhr_raw", "fhr_windows", "uc_windows", "static"]
    FEATURE_DIMS = {
        "uc_raw": FUSED_DIM,
        "fhr_raw": FUSED_DIM,
        "fhr_windows": FHR_WIN_DIM,
        "uc_windows": UC_WIN_DIM,
        "static": STATIC_DIM,
    }

    def to_feature_vec(m):
        parts = []
        for k in FEATURE_ORDER:
            v = np.asarray(m[k], dtype=np.float32).ravel()
            assert v.size == FEATURE_DIMS[k], f"{k} dim mismatch: {v.size} vs {FEATURE_DIMS[k]}"
            parts.append(v)
        x = np.concatenate(parts).astype(np.float32, copy=False)
        return x

    def make_xy(measurements):
        X = np.stack([to_feature_vec(m) for m in measurements], axis=0)
        y = np.asarray([m["target"] for m in measurements], dtype=np.float32).ravel()
        weeks = np.asarray([m["gest_age_weeks"] for m in measurements], dtype=np.int32)
        return X, y, weeks

    X_tr, y_tr, weeks_tr = make_xy(train_recs)
    X_te, y_te, weeks_te = make_xy(test_recs)

    print(f"[Fold {fold}] X_tr={X_tr.shape}, X_te={X_te.shape}")

    # --- 4. 训练 LGBM ---
    lgbm = LGBMRegressor(
        n_estimators=2000,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
    )

    lgbm.fit(X_tr, y_tr)

    # --- 5. 预测 & 评估 ---
    y_tr_pred = lgbm.predict(X_tr)
    y_te_pred = lgbm.predict(X_te)

    fold_train_mae = mean_absolute_error(y_tr, y_tr_pred)
    fold_test_mae = mean_absolute_error(y_te, y_te_pred)
    fold_test_rmse = np.sqrt(mean_squared_error(y_te, y_te_pred))
    fold_test_r2 = r2_score(y_te, y_te_pred)

    print(f"[Fold {fold}] Train MAE = {fold_train_mae:.3f}")
    print(f"[Fold {fold}] Test  MAE = {fold_test_mae:.3f}, RMSE = {fold_test_rmse:.3f}, R² = {fold_test_r2:.3f}")

    fold_metrics.append({
        "fold": fold,
        "train_mae": fold_train_mae,
        "test_mae": fold_test_mae,
        "test_rmse": fold_test_rmse,
        "test_r2": fold_test_r2,
    })

    # 记录全体散点用
    all_true.append(y_te)
    all_pred.append(y_te_pred)
    all_weeks_pred.append(weeks_te)

    # <<< NEW: 保存当前 fold 的所有 test 样本的 actual / predicted
    df_pred_fold = pd.DataFrame({
        "fold": fold,
        "gest_age_weeks": weeks_te.astype(int),
        "y_true": y_te.astype(float),
        "y_pred": y_te_pred.astype(float),
    })
    all_pred_rows.append(df_pred_fold)
    # >>> END NEW

    # --- 6. 每周 MAE ---
    abs_err = np.abs(y_te_pred - y_te)
    rows_this_fold = []
    print(f"[Fold {fold}] Test MAE by gestational week:")
    for w in sorted(np.unique(weeks_te)):
        mask_w = (weeks_te == w)
        n_w = int(mask_w.sum())
        if n_w == 0:
            continue
        mae_w = float(abs_err[mask_w].mean())
        rows_this_fold.append({
            "fold": fold,
            "week": int(w),
            "n_samples": n_w,
            "mae": mae_w,
        })
        print(f"  Week {int(w):2d}: MAE={mae_w:.3f} (n={n_w})")

    # 加到总表
    rows_week_fold.extend(rows_this_fold)

    # 每折单独保存一个 per-week CSV（可选）
    df_fold_week = pd.DataFrame(rows_this_fold)
    fold_week_csv = os.path.join(OUT_DIR, f"fold_{fold:02d}_week_mae.csv")
    df_fold_week.to_csv(fold_week_csv, index=False)
    print(f"[Saved] {fold_week_csv}")

# =======================
# 汇总 prediction.csv
# =======================

df_pred_all = pd.concat(all_pred_rows, ignore_index=True)   # <<< NEW
pred_csv_path = os.path.join(OUT_DIR, "prediction.csv")     # <<< NEW
df_pred_all.to_csv(pred_csv_path, index=False)              # <<< NEW
print(f"[Saved] {pred_csv_path}")                           # <<< NEW

# =======================
# 汇总结果
# =======================

all_true = np.concatenate(all_true)
all_pred = np.concatenate(all_pred)
all_weeks_pred = np.concatenate(all_weeks_pred)

# 每折整体性能表
df_folds = pd.DataFrame(fold_metrics)
fold_summary_csv = os.path.join(OUT_DIR, "fold_overall_metrics.csv")
df_folds.to_csv(fold_summary_csv, index=False)
print(f"[Saved] {fold_summary_csv}")

print("\nOverall across folds:")
print(df_folds)

# 每折 × 每周表
df_week_fold = pd.DataFrame(rows_week_fold)
week_fold_csv = os.path.join(OUT_DIR, "week_mae_per_fold.csv")
df_week_fold.to_csv(week_fold_csv, index=False)
print(f"[Saved] {week_fold_csv}")

# 按 week 汇总 mean / std
df_week_summary = df_week_fold.groupby("week").agg(
    mae_mean=("mae", "mean"),
    mae_std=("mae", "std"),
    n_total=("n_samples", "sum"),
    n_folds=("mae", "count"),
).reset_index()

week_summary_csv = os.path.join(OUT_DIR, "week_mae_summary.csv")
df_week_summary.to_csv(week_summary_csv, index=False)
print(f"[Saved] {week_summary_csv}")

print("\nWeek-wise MAE summary:")
print(df_week_summary)

# =======================
# 可视化 1：bar chart with SD (per week MAE)
# =======================

plt.figure(figsize=(10, 5))
x = df_week_summary["week"].values
y = df_week_summary["mae_mean"].values
yerr = df_week_summary["mae_std"].values

plt.bar(x, y, yerr=yerr, capsize=3)
plt.xlabel("Gestational Age (weeks)")
plt.ylabel("MAE (days-to-onset)")
plt.title("Per-week MAE across 5-fold CV (mean ± SD)")
plt.xticks(x)
plt.tight_layout()
bar_path = os.path.join(OUT_DIR, "week_mae_bar_with_sd.png")
plt.savefig(bar_path, dpi=200)
plt.close()
print(f"[Saved] {bar_path}")

# =======================
# 可视化 2：拟合散点图 (true vs pred)
# =======================

plt.figure(figsize=(6, 6))
plt.scatter(all_true, all_pred, s=8, alpha=0.5)
mn = float(min(all_true.min(), all_pred.min()))
mx = float(max(all_true.max(), all_pred.max()))
plt.plot([mn, mx], [mn, mx], "r--", label="Ideal")

overall_mae = mean_absolute_error(all_true, all_pred)
overall_rmse = np.sqrt(mean_squared_error(all_true, all_pred))
overall_r2 = r2_score(all_true, all_pred)

plt.xlabel("True days to onset")
plt.ylabel("Predicted days to onset")
plt.title(f"5-fold CV — True vs Pred\nMAE={overall_mae:.2f}, RMSE={overall_rmse:.2f}, R²={overall_r2:.3f}")
plt.legend(loc="upper left")
plt.tight_layout()
scatter_path = os.path.join(OUT_DIR, "cv_true_vs_pred_scatter.png")
plt.savefig(scatter_path, dpi=200)
plt.close()
print(f"[Saved] {scatter_path}")


===== Fold 1/5 =====
Train : 2892 from 45 patients
Test  : 769 from 12 patients

[Fold 1] Train encoder for uc_raw
[uc_raw] Epoch 001 | Loss = 697.7561
[uc_raw] Epoch 002 | Loss = 411.7242
[uc_raw] Epoch 003 | Loss = 389.0933
[uc_raw] Epoch 004 | Loss = 370.7957
[uc_raw] Epoch 005 | Loss = 351.3145
[uc_raw] Epoch 006 | Loss = 332.0038
[uc_raw] Epoch 007 | Loss = 325.1339
[uc_raw] Epoch 008 | Loss = 315.4523
[uc_raw] Epoch 009 | Loss = 309.9812
[uc_raw] Epoch 010 | Loss = 300.9370
[uc_raw] Epoch 011 | Loss = 284.9339
[uc_raw] Epoch 012 | Loss = 265.7664
[uc_raw] Epoch 013 | Loss = 252.2780
[uc_raw] Epoch 014 | Loss = 230.2932
[uc_raw] Epoch 015 | Loss = 221.4088
[uc_raw] Epoch 016 | Loss = 185.8327
[uc_raw] Epoch 017 | Loss = 181.3918
[uc_raw] Epoch 018 | Loss = 156.7488
[uc_raw] Epoch 019 | Loss = 137.2236
[uc_raw] Epoch 020 | Loss = 136.1366
[uc_raw] Epoch 021 | Loss = 127.8324
[uc_raw] Epoch 022 | Loss = 111.5219
[uc_raw] Epoch 023 | Loss = 100.7470
[uc_raw] Epoch 024 | Loss = 93.04

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[Fold 1] Train MAE = 0.043
[Fold 1] Test  MAE = 5.340, RMSE = 7.090, R² = 0.826
[Fold 1] Test MAE by gestational week:
  Week 31: MAE=3.933 (n=8)
  Week 32: MAE=5.223 (n=29)
  Week 33: MAE=4.329 (n=61)
  Week 34: MAE=5.964 (n=69)
  Week 35: MAE=5.544 (n=89)
  Week 36: MAE=7.405 (n=112)
  Week 37: MAE=6.477 (n=113)
  Week 38: MAE=4.744 (n=100)
  Week 39: MAE=4.937 (n=107)
  Week 40: MAE=2.600 (n=67)
  Week 41: MAE=1.170 (n=14)
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_01_week_mae.csv

===== Fold 2/5 =====
Train : 2903 from 45 patients
Test  : 758 from 12 patients

[Fold 2] Train encoder for uc_raw
[uc_raw] Epoch 001 | Loss = 696.6121
[uc_raw] Epoch 002 | Loss = 376.3167
[uc_raw] Epoch 003 | Loss = 353.2174
[uc_raw] Epoch 004 | Loss = 329.9570
[uc_raw] Epoch 005 | Loss = 317.0782
[uc_raw] Epoch 006 | Loss = 310.3385
[uc_raw] Epoch 007 | Loss = 300.9177
[uc_raw] Epoch 008 | Loss = 294.3403
[uc_raw] Epoch 009 | Loss = 291.5218
[uc_raw] Epoch 010 | Loss = 282.3551
[uc_raw] Epoch 

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[Fold 2] Train MAE = 0.043
[Fold 2] Test  MAE = 4.449, RMSE = 5.370, R² = 0.931
[Fold 2] Test MAE by gestational week:
  Week 29: MAE=3.221 (n=7)
  Week 30: MAE=5.638 (n=29)
  Week 31: MAE=6.669 (n=35)
  Week 32: MAE=5.518 (n=35)
  Week 33: MAE=6.264 (n=43)
  Week 34: MAE=6.022 (n=58)
  Week 35: MAE=5.370 (n=63)
  Week 36: MAE=4.741 (n=87)
  Week 37: MAE=3.989 (n=95)
  Week 38: MAE=3.063 (n=118)
  Week 39: MAE=3.881 (n=112)
  Week 40: MAE=2.971 (n=71)
  Week 41: MAE=0.939 (n=5)
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_02_week_mae.csv

===== Fold 3/5 =====
Train : 2847 from 46 patients
Test  : 814 from 11 patients

[Fold 3] Train encoder for uc_raw
[uc_raw] Epoch 001 | Loss = 618.4390
[uc_raw] Epoch 002 | Loss = 380.8885
[uc_raw] Epoch 003 | Loss = 360.8768
[uc_raw] Epoch 004 | Loss = 348.0297
[uc_raw] Epoch 005 | Loss = 330.2261
[uc_raw] Epoch 006 | Loss = 318.1002
[uc_raw] Epoch 007 | Loss = 310.4290
[uc_raw] Epoch 008 | Loss = 306.0653
[uc_raw] Epoch 009 | Loss = 296.5803

...

[Fold 3] Train MAE = 0.052
[Fold 3] Test  MAE = 5.172, RMSE = 6.329, R² = 0.898
[Fold 3] Test MAE by gestational week:
  Week 29: MAE=13.848 (n=14)
  Week 30: MAE=7.243 (n=11)
  Week 31: MAE=2.202 (n=15)
  Week 32: MAE=1.987 (n=21)
  Week 33: MAE=6.100 (n=60)
  Week 34: MAE=6.683 (n=69)
  Week 35: MAE=5.956 (n=87)
  Week 36: MAE=6.043 (n=112)
  Week 37: MAE=5.784 (n=122)
  Week 38: MAE=4.893 (n=117)
  Week 39: MAE=3.935 (n=109)
  Week 40: MAE=1.873 (n=65)
  Week 41: MAE=0.915 (n=12)
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_03_week_mae.csv

===== Fold 4/5 =====
Train : 3072 from 46 patients
Test  : 589 from 11 patients

[Fold 4] Train encoder for uc_raw
[uc_raw] Epoch 001 | Loss = 679.6620
[uc_raw] Epoch 002 | Loss = 403.1255
[uc_raw] Epoch 003 | Loss = 378.5281
[uc_raw] Epoch 004 | Loss = 350.0578
[uc_raw] Epoch 005 | Loss = 330.8941
[uc_raw] Epoch 006 | Loss = 319.4835
[uc_raw] Epoch 007 | Loss = 301.1830
[uc_raw] Epoch 008 | Loss = 286.9255
[uc_raw] Epoch 009 | Loss = 271

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[Fold 4] Train MAE = 0.027
[Fold 4] Test  MAE = 9.406, RMSE = 12.082, R² = 0.541
[Fold 4] Test MAE by gestational week:
  Week 30: MAE=28.484 (n=26)
  Week 31: MAE=17.491 (n=33)
  Week 32: MAE=12.392 (n=32)
  Week 33: MAE=13.975 (n=37)
  Week 34: MAE=7.093 (n=48)
  Week 35: MAE=8.281 (n=73)
  Week 36: MAE=9.245 (n=61)
  Week 37: MAE=8.492 (n=93)
  Week 38: MAE=6.848 (n=99)
  Week 39: MAE=4.872 (n=54)
  Week 40: MAE=2.330 (n=25)
  Week 41: MAE=1.358 (n=8)
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_04_week_mae.csv

===== Fold 5/5 =====
Train : 2930 from 46 patients
Test  : 731 from 11 patients

[Fold 5] Train encoder for uc_raw
[uc_raw] Epoch 001 | Loss = 718.2447
[uc_raw] Epoch 002 | Loss = 426.2859
[uc_raw] Epoch 003 | Loss = 391.4116
[uc_raw] Epoch 004 | Loss = 364.7944
[uc_raw] Epoch 005 | Loss = 345.9159
[uc_raw] Epoch 006 | Loss = 332.1648
[uc_raw] Epoch 007 | Loss = 318.4395
[uc_raw] Epoch 008 | Loss = 312.1371
[uc_raw] Epoch 009 | Loss = 303.4573
[uc_raw] Epoch 010 | Lo

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[Fold 5] Train MAE = 0.041
[Fold 5] Test  MAE = 9.107, RMSE = 12.482, R² = 0.462
[Fold 5] Test MAE by gestational week:
  Week 29: MAE=16.393 (n=4)
  Week 30: MAE=14.779 (n=6)
  Week 31: MAE=28.449 (n=14)
  Week 32: MAE=19.185 (n=40)
  Week 33: MAE=10.979 (n=53)
  Week 34: MAE=8.790 (n=69)
  Week 35: MAE=8.692 (n=59)
  Week 36: MAE=9.449 (n=71)
  Week 37: MAE=8.531 (n=80)
  Week 38: MAE=8.930 (n=110)
  Week 39: MAE=7.078 (n=81)
  Week 40: MAE=5.762 (n=93)
  Week 41: MAE=3.760 (n=50)
  Week 42: MAE=3.323 (n=1)
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_05_week_mae.csv
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/prediction.csv
[Saved] /content/drive/MyDrive/ts2vec-05-yuntao/fold_overall_metrics.csv

Overall across folds:
   fold  train_mae  test_mae  test_rmse   test_r2
0     1   0.043475  5.339773   7.089605  0.825643
1     2   0.043002  4.449458   5.369974  0.931447
2     3   0.051883  5.172239   6.328985  0.897702
3     4   0.026547  9.405993  12.081721  0.540674
4     5

## add


In [ ]:
import os
import json
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pywt

# =======================
# Config
# =======================
target_var = "add"   # 'add', 'onset', or 'hist'

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

BATCH_SIZE_ENC = 64       # CNN+BiLSTM encoder batch size
EPOCHS_ENC = 40           # encoder training epochs
LR_ENC = 1e-3             # encoder learning rate

D_MODEL = 64              # CNN/BiLSTM feature dim
GA_EMB_DIM = 16           # GA MLP embedding dim
DROPOUT = 0.20

WAVELET_NAME = "db4"
WAVELET_LEVEL = 3         # raw + D1..D3 + A3 = 5 channels

N_FOLDS = 5

OUT_DIR = r"F:\E3A Healthcare\TS2VEC+LGBM\cv_swt_cnn_bilstm_ga_lgbm_5fold_add"
os.makedirs(OUT_DIR, exist_ok=True)

np.random.seed(SEED)
torch.manual_seed(SEED)

# =======================
# Load data
# =======================

DATA_ROOT = r"F:\E3A Healthcare\TS2VEC+LGBM\datasets"
json_path = os.path.join(DATA_ROOT, f"dataset_{target_var}.json")

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame.from_records(data)
print(len(df), "Measurements (raw)")

# =======================
# Basic cleaning & static imputation
# =======================

age_mean = np.mean([i[0] for i in df["static"] if pd.notna(i[0])])
bmi_mean = np.mean([i[1] for i in df["static"] if pd.notna(i[1])])

cleaned_data = []
for _, m in enumerate(data):

    # Remove measurements with empty windows
    if len(m["uc_windows"]) == 0 or len(m["fhr_windows"]) == 0:
        continue

    # Handle NaN values in static
    static = m["static"].copy()
    if pd.isna(static[0]):
        static[0] = age_mean
    if pd.isna(static[1]):
        static[1] = bmi_mean

    copy_m = m.copy()
    copy_m["static"] = static
    cleaned_data.append(copy_m)

cleaned_df = pd.DataFrame(cleaned_data)
print(len(cleaned_df), "Cleaned Measurements")

# 假定 static 最后一个元素是 gest_age_days
cleaned_df["gest_age_weeks"] = [(i[-1] // 7) + 1 for i in cleaned_df["static"]]

base_records = cleaned_df.to_dict(orient="records")  # 原始不带任何 fold 修改的版本
N = len(base_records)
all_weeks = cleaned_df["gest_age_weeks"].to_numpy()

print("Gestational age weeks distribution:")
print(cleaned_df["gest_age_weeks"].value_counts().sort_index())

# =======================
# SWT helper
# =======================

def sproc_wavelet_channels(x_1d: np.ndarray):
    """
    SWT(db4, level=3) -> 通道: [raw, D1, D2, D3, A3]
    """
    x_1d = np.asarray(x_1d, dtype=np.float32)
    coeffs = pywt.swt(
        x_1d, wavelet=WAVELET_NAME,
        level=WAVELET_LEVEL,
        trim_approx=False,
        norm=True
    )
    # coeffs[l] = (cA_l, cD_l)
    chans = [x_1d.astype(np.float32)]  # raw
    for l in range(WAVELET_LEVEL):
        cA, cD = coeffs[l]
        chans.append(cD.astype(np.float32))   # D1..D3
    chans.append(coeffs[-1][0].astype(np.float32))  # A3
    X = np.stack(chans, axis=0)  # (C, T)

    # 逐通道 robust 标准化
    X = (X - np.median(X, axis=1, keepdims=True)) / (
        np.std(X, axis=1, keepdims=True) + 1e-6
    )
    return X.astype(np.float32)

# =======================
# CNN+BiLSTM + GA encoder
# =======================

class SeqGADataset(Dataset):
    """
    Dataset for single signal (uc_raw or fhr_raw) + GA + target
    在 __getitem__ 里做 SWT，输出 (C, T)。
    """
    def __init__(self, measurements, signal_key):
        self.measurements = measurements
        self.signal_key = signal_key

    def __len__(self):
        return len(self.measurements)

    def __getitem__(self, idx):
        m = self.measurements[idx]

        seq = np.asarray(m[self.signal_key], dtype=np.float32)  # (T,)
        x_ch = sproc_wavelet_channels(seq)                       # (C, T)

        y = np.float32(m["target"])
        ga = np.float32(m["gest_age_weeks"])  # gest age in weeks

        return torch.from_numpy(x_ch), torch.tensor(y), torch.tensor(ga)


class ConvBlock(nn.Module):
    def __init__(self, c_in, c_out, k, d=1, p=0.0, groups_gn=8):
        super().__init__()
        pad = (k - 1) // 2 * d
        self.conv = nn.Conv1d(
            c_in, c_out, kernel_size=k, dilation=d,
            padding=pad, bias=False
        )
        self.gn = nn.GroupNorm(num_groups=min(groups_gn, c_out),
                               num_channels=c_out)
        self.act = nn.SiLU()
        self.dp = nn.Dropout(p)
        self.res = (c_in == c_out)

    def forward(self, x):
        y = self.dp(self.act(self.gn(self.conv(x))))
        return y + x if self.res else y


class CNNBiLSTM_GA_Encoder(nn.Module):
    """
    结构参考你原来的 CNNBiLSTM_GA_Reg：
    - stem Conv1d
    - 多层 ConvBlock (dilation k=9,7,5, d=1,2,4)
    - BiLSTM
    - GA MLP
    - 输出:
        pred: (B,)
        feat: (B, D_MODEL + GA_EMB_DIM)
    输入:
        x:  (B, C, T)  SWT 通道
        ga: (B,)
    """
    def __init__(self, in_ch=5, d_model=64, ga_emb_dim=16, dropout=0.2):
        super().__init__()
        self.d_model = d_model
        self.ga_emb_dim = ga_emb_dim

        self.stem = nn.Sequential(
            nn.Conv1d(in_ch, d_model, kernel_size=7, padding=3, bias=False),
            nn.GroupNorm(num_groups=8, num_channels=d_model),
            nn.SiLU(),
        )

        self.cnn_blocks = nn.Sequential(
            ConvBlock(d_model, d_model, k=9, d=1, p=dropout),
            ConvBlock(d_model, d_model, k=7, d=2, p=dropout),
            ConvBlock(d_model, d_model, k=5, d=4, p=dropout),
            nn.Conv1d(d_model, d_model, kernel_size=1, bias=False),
            nn.SiLU(),
            nn.Dropout(dropout),
        )

        self.bilstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model // 2,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )

        self.pre_pool_dp = nn.Dropout(dropout)

        self.ga_mlp = nn.Sequential(
            nn.Linear(1, 16),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(16, ga_emb_dim),
            nn.SiLU(),
        )

        fuse_dim = d_model + ga_emb_dim
        self.head = nn.Sequential(
            nn.Linear(fuse_dim, fuse_dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(fuse_dim, 1),
        )

    def forward(self, x, ga_scalar):
        """
        x:  (B, C, T)
        ga_scalar: (B,)
        """
        f = self.stem(x)            # (B, d_model, T)
        f = self.cnn_blocks(f)      # (B, d_model, T)

        # BiLSTM
        f_t = f.transpose(1, 2)     # (B, T, d_model)
        y, _ = self.bilstm(f_t)     # (B, T, d_model)
        y = self.pre_pool_dp(y)

        # 时间维 average pooling
        feat_seq = y.mean(dim=1)    # (B, d_model)

        # GA embedding
        ga_emb = self.ga_mlp(ga_scalar.unsqueeze(1))  # (B, ga_emb_dim)

        fused = torch.cat([feat_seq, ga_emb], dim=1)  # (B, d_model + ga_emb_dim)
        pred = self.head(fused).squeeze(-1)           # (B,)

        return pred, fused


def train_ga_encoder(measurements, signal_key):
    """
    在 train 集上训练一个 SWT+CNN+BiLSTM+GA encoder（监督回归），返回训练好的模型。
    """
    dataset = SeqGADataset(measurements, signal_key)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE_ENC,
                        shuffle=True, drop_last=False)

    model = CNNBiLSTM_GA_Encoder(
        in_ch=5,
        d_model=D_MODEL,
        ga_emb_dim=GA_EMB_DIM,
        dropout=DROPOUT,
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR_ENC)
    loss_fn = nn.MSELoss()

    model.train()
    for epoch in range(1, EPOCHS_ENC + 1):
        epoch_loss = 0.0
        n_samples = 0
        for x_ch, y, ga in loader:
            x_ch = x_ch.to(DEVICE)      # (B, C, T)
            y = y.to(DEVICE)            # (B,)
            ga = ga.to(DEVICE)          # (B,)

            optimizer.zero_grad()
            pred, _ = model(x_ch, ga)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()

            bs = x_ch.size(0)
            epoch_loss += loss.item() * bs
            n_samples += bs

        print(f"[{signal_key}] Epoch {epoch:03d} | Loss = {epoch_loss / n_samples:.4f}")

    return model


def extract_ga_features(measurements, signal_key, model):
    """
    用训练好的 encoder 抽取融合 GA 的特征。
    返回 (N, d_model + ga_emb_dim) 的 numpy 数组。
    """
    dataset = SeqGADataset(measurements, signal_key)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE_ENC,
                        shuffle=False, drop_last=False)

    model.eval()
    feats = []
    with torch.no_grad():
        for x_ch, y, ga in loader:
            x_ch = x_ch.to(DEVICE)       # (B, C, T)
            ga = ga.to(DEVICE)
            _, feat = model(x_ch, ga)    # (B, d_model + ga_emb_dim)
            feats.append(feat.cpu().numpy())

    feats = np.concatenate(feats, axis=0)
    return feats

# =======================
# 5-fold cross-validation
# =======================

kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

all_true = []
all_pred = []
all_weeks_pred = []
fold_metrics = []
rows_week_fold = []
all_pred_rows = []   # <<< NEW: 用于保存所有 test 样本的预测

indices = np.arange(N)

for fold, (tr_idx, te_idx) in enumerate(kf.split(indices), start=1):
    print(f"\n===== Fold {fold}/{N_FOLDS} =====")

    # 每个 fold 用 base_records 的深拷贝，避免之前的修改污染
    train_recs = [copy.deepcopy(base_records[i]) for i in tr_idx]
    test_recs = [copy.deepcopy(base_records[i]) for i in te_idx]

    print(f"Train samples: {len(train_recs)}, Test samples: {len(test_recs)}")

    # --- 1. 训练两个 encoder ---
    print("\n[Fold %d] Train encoder for uc_raw" % fold)
    encoder_uc = train_ga_encoder(train_recs, "uc_raw")

    print("\n[Fold %d] Train encoder for fhr_raw" % fold)
    encoder_fhr = train_ga_encoder(train_recs, "fhr_raw")

    # --- 2. 提取 embedding ---
    print("\n[Fold %d] Extract embeddings (uc_raw & fhr_raw)" % fold)
    train_uc_embed = extract_ga_features(train_recs, "uc_raw", encoder_uc)
    train_fhr_embed = extract_ga_features(train_recs, "fhr_raw", encoder_fhr)
    test_uc_embed = extract_ga_features(test_recs, "uc_raw", encoder_uc)
    test_fhr_embed = extract_ga_features(test_recs, "fhr_raw", encoder_fhr)

    for i, e in enumerate(train_uc_embed):
        train_recs[i]["uc_raw"] = e
    for i, e in enumerate(train_fhr_embed):
        train_recs[i]["fhr_raw"] = e
    for i, e in enumerate(test_uc_embed):
        test_recs[i]["uc_raw"] = e
    for i, e in enumerate(test_fhr_embed):
        test_recs[i]["fhr_raw"] = e

    # --- 3. 聚合 window-level 特征 ---
    for rec in train_recs:
        uc_ws = rec["uc_windows"]
        fhr_ws = rec["fhr_windows"]
        # 原始是 list[dict]；这里才做 mean pooling
        uc_w = np.array([[v for _, v in w.items()] for w in uc_ws],
                        dtype=np.float32)
        fhr_w = np.array([[v for _, v in w.items()] for w in fhr_ws],
                         dtype=np.float32)
        rec["uc_windows"] = uc_w.mean(axis=0)
        rec["fhr_windows"] = fhr_w.mean(axis=0)

    for rec in test_recs:
        uc_ws = rec["uc_windows"]
        fhr_ws = rec["fhr_windows"]
        uc_w = np.array([[v for _, v in w.items()] for w in uc_ws],
                        dtype=np.float32)
        fhr_w = np.array([[v for _, v in w.items()] for w in fhr_ws],
                         dtype=np.float32)
        rec["uc_windows"] = uc_w.mean(axis=0)
        rec["fhr_windows"] = fhr_w.mean(axis=0)

    # 维度信息
    FUSED_DIM = len(train_recs[0]["uc_raw"])        # 64 + 16 = 80
    STATIC_DIM = len(train_recs[0]["static"])
    UC_WIN_DIM = len(train_recs[0]["uc_windows"])   # 20
    FHR_WIN_DIM = len(train_recs[0]["fhr_windows"]) # 24

    FEATURE_ORDER = ["uc_raw", "fhr_raw", "fhr_windows", "uc_windows", "static"]
    FEATURE_DIMS = {
        "uc_raw": FUSED_DIM,
        "fhr_raw": FUSED_DIM,
        "fhr_windows": FHR_WIN_DIM,
        "uc_windows": UC_WIN_DIM,
        "static": STATIC_DIM,
    }

    def to_feature_vec(m):
        parts = []
        for k in FEATURE_ORDER:
            v = np.asarray(m[k], dtype=np.float32).ravel()
            assert v.size == FEATURE_DIMS[k], f"{k} dim mismatch: {v.size} vs {FEATURE_DIMS[k]}"
            parts.append(v)
        x = np.concatenate(parts).astype(np.float32, copy=False)
        return x

    def make_xy(measurements):
        X = np.stack([to_feature_vec(m) for m in measurements], axis=0)
        y = np.asarray([m["target"] for m in measurements], dtype=np.float32).ravel()
        weeks = np.asarray([m["gest_age_weeks"] for m in measurements], dtype=np.int32)
        return X, y, weeks

    X_tr, y_tr, weeks_tr = make_xy(train_recs)
    X_te, y_te, weeks_te = make_xy(test_recs)

    print(f"[Fold {fold}] X_tr={X_tr.shape}, X_te={X_te.shape}")

    # --- 4. 训练 LGBM ---
    lgbm = LGBMRegressor(
        n_estimators=2000,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
    )

    lgbm.fit(X_tr, y_tr)

    # --- 5. 预测 & 评估 ---
    y_tr_pred = lgbm.predict(X_tr)
    y_te_pred = lgbm.predict(X_te)

    fold_train_mae = mean_absolute_error(y_tr, y_tr_pred)
    fold_test_mae = mean_absolute_error(y_te, y_te_pred)
    fold_test_rmse = np.sqrt(mean_squared_error(y_te, y_te_pred))
    fold_test_r2 = r2_score(y_te, y_te_pred)

    print(f"[Fold {fold}] Train MAE = {fold_train_mae:.3f}")
    print(f"[Fold {fold}] Test  MAE = {fold_test_mae:.3f}, RMSE = {fold_test_rmse:.3f}, R² = {fold_test_r2:.3f}")

    fold_metrics.append({
        "fold": fold,
        "train_mae": fold_train_mae,
        "test_mae": fold_test_mae,
        "test_rmse": fold_test_rmse,
        "test_r2": fold_test_r2,
    })

    # 记录全体散点用
    all_true.append(y_te)
    all_pred.append(y_te_pred)
    all_weeks_pred.append(weeks_te)

    # <<< NEW: 保存当前 fold 的所有 test 样本的 actual / predicted
    df_pred_fold = pd.DataFrame({
        "fold": fold,
        "gest_age_weeks": weeks_te.astype(int),
        "y_true": y_te.astype(float),
        "y_pred": y_te_pred.astype(float),
    })
    all_pred_rows.append(df_pred_fold)
    # >>> END NEW

    # --- 6. 每周 MAE ---
    abs_err = np.abs(y_te_pred - y_te)
    rows_this_fold = []
    print(f"[Fold {fold}] Test MAE by gestational week:")
    for w in sorted(np.unique(weeks_te)):
        mask_w = (weeks_te == w)
        n_w = int(mask_w.sum())
        if n_w == 0:
            continue
        mae_w = float(abs_err[mask_w].mean())
        rows_this_fold.append({
            "fold": fold,
            "week": int(w),
            "n_samples": n_w,
            "mae": mae_w,
        })
        print(f"  Week {int(w):2d}: MAE={mae_w:.3f} (n={n_w})")

    # 加到总表
    rows_week_fold.extend(rows_this_fold)

    # 每折单独保存一个 per-week CSV（可选）
    df_fold_week = pd.DataFrame(rows_this_fold)
    fold_week_csv = os.path.join(OUT_DIR, f"fold_{fold:02d}_week_mae.csv")
    df_fold_week.to_csv(fold_week_csv, index=False)
    print(f"[Saved] {fold_week_csv}")

# =======================
# 汇总 prediction.csv
# =======================

df_pred_all = pd.concat(all_pred_rows, ignore_index=True)   # <<< NEW
pred_csv_path = os.path.join(OUT_DIR, "prediction.csv")     # <<< NEW
df_pred_all.to_csv(pred_csv_path, index=False)              # <<< NEW
print(f"[Saved] {pred_csv_path}")                           # <<< NEW

# =======================
# 汇总结果
# =======================

all_true = np.concatenate(all_true)
all_pred = np.concatenate(all_pred)
all_weeks_pred = np.concatenate(all_weeks_pred)

# 每折整体性能表
df_folds = pd.DataFrame(fold_metrics)
fold_summary_csv = os.path.join(OUT_DIR, "fold_overall_metrics.csv")
df_folds.to_csv(fold_summary_csv, index=False)
print(f"[Saved] {fold_summary_csv}")

print("\nOverall across folds:")
print(df_folds)

# 每折 × 每周表
df_week_fold = pd.DataFrame(rows_week_fold)
week_fold_csv = os.path.join(OUT_DIR, "week_mae_per_fold.csv")
df_week_fold.to_csv(week_fold_csv, index=False)
print(f"[Saved] {week_fold_csv}")

# 按 week 汇总 mean / std
df_week_summary = df_week_fold.groupby("week").agg(
    mae_mean=("mae", "mean"),
    mae_std=("mae", "std"),
    n_total=("n_samples", "sum"),
    n_folds=("mae", "count"),
).reset_index()

week_summary_csv = os.path.join(OUT_DIR, "week_mae_summary.csv")
df_week_summary.to_csv(week_summary_csv, index=False)
print(f"[Saved] {week_summary_csv}")

print("\nWeek-wise MAE summary:")
print(df_week_summary)

# =======================
# 可视化 1：bar chart with SD (per week MAE)
# =======================

plt.figure(figsize=(10, 5))
x = df_week_summary["week"].values
y = df_week_summary["mae_mean"].values
yerr = df_week_summary["mae_std"].values

plt.bar(x, y, yerr=yerr, capsize=3)
plt.xlabel("Gestational Age (weeks)")
plt.ylabel("MAE (days-to-onset)")
plt.title("Per-week MAE across 5-fold CV (mean ± SD)")
plt.xticks(x)
plt.tight_layout()
bar_path = os.path.join(OUT_DIR, "week_mae_bar_with_sd.png")
plt.savefig(bar_path, dpi=200)
plt.close()
print(f"[Saved] {bar_path}")

# =======================
# 可视化 2：拟合散点图 (true vs pred)
# =======================

plt.figure(figsize=(6, 6))
plt.scatter(all_true, all_pred, s=8, alpha=0.5)
mn = float(min(all_true.min(), all_pred.min()))
mx = float(max(all_true.max(), all_pred.max()))
plt.plot([mn, mx], [mn, mx], "r--", label="Ideal")

overall_mae = mean_absolute_error(all_true, all_pred)
overall_rmse = np.sqrt(mean_squared_error(all_true, all_pred))
overall_r2 = r2_score(all_true, all_pred)

plt.xlabel("True days to onset")
plt.ylabel("Predicted days to onset")
plt.title(f"5-fold CV — True vs Pred\nMAE={overall_mae:.2f}, RMSE={overall_rmse:.2f}, R²={overall_r2:.3f}")
plt.legend(loc="upper left")
plt.tight_layout()
scatter_path = os.path.join(OUT_DIR, "cv_true_vs_pred_scatter.png")
plt.savefig(scatter_path, dpi=200)
plt.close()
print(f"[Saved] {scatter_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\E3A Healthcare\\TS2VEC+LGBM\\datasets/dataset_add.json'